In [2]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
# functions
def enum_row(row):
    print(row['state'])


def find_state_code(row):
    if row['state'] != 0:
        print(process.extractOne(row['state'], states, score_cutoff=80))


def capital(str):
    return str.capitalize()


def correct_state(row):
    if row['state'] != 0:
        state = process.extractOne(row['state'], states, score_cutoff=80)
        if state:
            state_name = state[0]
            return ' '.join(map(capital, state_name.split(' ')))
    return row['state']


def fill_state_code(row):
    if row['state'] != 0:
        state = process.extractOne(row['state'], states, score_cutoff=80)
        if state:
            state_name = state[0]
            return state_to_code[state_name]
    return ''

In [6]:
# starting main
pd.set_option('display.width', 200)
data = pd.read_excel('.\\sales.xlsx', sheetname='sheet1', header=0)
print('data.head() = \n', data.head())
print('data.tail() = \n', data.tail())
print('data.dtypes = \n', data.dtypes)
print('data.columns = \n', data.columns)

for c in data.columns:
    print(c, end=' ')

print()
data['total'] = data['Jan'] + data['Feb'] + data['Mar']
print(data.head())
print(data['Jan'].sum())
print(data['Jan'].min())
print(data['Jan'].max())
print(data['Jan'].mean())

print('=============')

data.head() = 
    account                         name                                street             city          state  postal-code     Jan     Feb    Mar
0   211829   Kerluke, Koepp and Hilpert                    34456 Sean Highway       New Jaycob          Texas        28752   10000   62000  35000
1   320563               Walter-Trantow                     1311 Alvis Tunnel    Port Khadijah  NorthCarolina        38365   95000   45000  35000
2   648336   Bashirian, Kunde and Price  62184 Schamberger Underpass Apt. 231   New Lilianland           Iowa        76517   91000  120000  35000
3   109996  D'Amore, Gleichner and Bode           155 Fadel Crescent Apt. 144       Hyattburgh          Maine        46021   45000  120000  10000
4   121213                Bauch-Goldner                   7274 Marissa Common  Shanahanchester     California        49681  162000  120000  35000
data.tail() = 
     account                          name                     street             city       

In [7]:
# 添加一行
s1 = data[['Jan', 'Feb', 'Mar', 'total']].sum()
print(s1)
s2 = pd.DataFrame(data=s1)
print(s2)
print(s2.T)
print(s2.T.reindex(columns=data.columns))
# 即：
s = pd.DataFrame(data=data[['Jan', 'Feb', 'Mar', 'total']].sum()).T
s = s.reindex(columns=data.columns, fill_value=0)
print(s)
data = data.append(s, ignore_index=True)
data = data.rename(index={15:'Total'})
print(data.tail())

Jan      1462000
Feb      1507000
Mar       717000
total    3686000
dtype: int64
             0
Jan    1462000
Feb    1507000
Mar     717000
total  3686000
       Jan      Feb     Mar    total
0  1462000  1507000  717000  3686000
   account  name  street  city  state  postal-code      Jan      Feb     Mar    total
0      NaN   NaN     NaN   NaN    NaN          NaN  1462000  1507000  717000  3686000
   account  name  street  city  state  postal-code      Jan      Feb     Mar    total
0        0     0       0     0      0            0  1462000  1507000  717000  3686000
       account                          name                    street             city        state  postal-code      Jan      Feb     Mar    total
11      231907                    Hahn-Moore  18115 Olivine Throughway    Norbertomouth  NorthDakota        31415   150000    10000  162000   322000
12      242368  Frami, Anderson and Donnelly           182 Bertie Road      East Davian         Iowa        72686   162000   120

In [9]:
# apply的使用
print('==============apply的使用==========')
data.apply(enum_row, axis=1)

state_to_code = {"VERMONT": "VT", "GEORGIA": "GA", "IOWA": "IA", "Armed Forces Pacific": "AP", "GUAM": "GU",
                 "KANSAS": "KS", "FLORIDA": "FL", "AMERICAN SAMOA": "AS", "NORTH CAROLINA": "NC", "HAWAII": "HI",
                 "NEW YORK": "NY", "CALIFORNIA": "CA", "ALABAMA": "AL", "IDAHO": "ID",
                 "FEDERATED STATES OF MICRONESIA": "FM",
                 "Armed Forces Americas": "AA", "DELAWARE": "DE", "ALASKA": "AK", "ILLINOIS": "IL",
                 "Armed Forces Africa": "AE", "SOUTH DAKOTA": "SD", "CONNECTICUT": "CT", "MONTANA": "MT",
                 "MASSACHUSETTS": "MA",
                 "PUERTO RICO": "PR", "Armed Forces Canada": "AE", "NEW HAMPSHIRE": "NH", "MARYLAND": "MD",
                 "NEW MEXICO": "NM",
                 "MISSISSIPPI": "MS", "TENNESSEE": "TN", "PALAU": "PW", "COLORADO": "CO",
                 "Armed Forces Middle East": "AE",
                 "NEW JERSEY": "NJ", "UTAH": "UT", "MICHIGAN": "MI", "WEST VIRGINIA": "WV", "WASHINGTON": "WA",
                 "MINNESOTA": "MN", "OREGON": "OR", "VIRGINIA": "VA", "VIRGIN ISLANDS": "VI",
                 "MARSHALL ISLANDS": "MH",
                 "WYOMING": "WY", "OHIO": "OH", "SOUTH CAROLINA": "SC", "INDIANA": "IN", "NEVADA": "NV",
                 "LOUISIANA": "LA",
                 "NORTHERN MARIANA ISLANDS": "MP", "NEBRASKA": "NE", "ARIZONA": "AZ", "WISCONSIN": "WI",
                 "NORTH DAKOTA": "ND",
                 "Armed Forces Europe": "AE", "PENNSYLVANIA": "PA", "OKLAHOMA": "OK", "KENTUCKY": "KY",
                 "RHODE ISLAND": "RI",
                 "DISTRICT OF COLUMBIA": "DC", "ARKANSAS": "AR", "MISSOURI": "MO", "TEXAS": "TX", "MAINE": "ME"}
states = list(state_to_code.keys())
print(fuzz.ratio('Python Package', 'PythonPackage'))
print(process.extract('Mississippi', states))
print(process.extract('Mississipi', states, limit=1))
print(process.extractOne('Mississipi', states))
data.apply(find_state_code, axis=1)

print('Before Correct State:\n', data['state'])
data['state'] = data.apply(correct_state, axis=1)
print('After Correct State:\n', data['state'])
data.insert(5, 'State Code', np.nan)
data['State Code'] = data.apply(fill_state_code, axis=1)
print(data)

# group by
print('==============group by================')
print(data.groupby('State Code'))
print('All Columns:\n')
print(data.groupby('State Code').sum())
print('Short Columns:\n')
print(data[['State Code', 'Jan', 'Feb', 'Mar', 'total']].groupby('State Code').sum())

# 写入文件
data.to_excel('sales_result.xls', sheet_name='Sheet1', index=False)

==============apply的使用==========
Texas
NorthCarolina
Iowa
Maine
California
Arkansas
Mississipi
RhodeIsland
Pennsylvania
Idaho
Tenessee
NorthDakota
Iowa
RhodeIsland
Delaware
0
96
[('MISSISSIPPI', 100), ('MISSOURI', 53), ('MASSACHUSETTS', 42), ('WISCONSIN', 40), ('VIRGIN ISLANDS', 38)]
[('MISSISSIPPI', 95)]
('MISSISSIPPI', 95)
('TEXAS', 100)
('NORTH CAROLINA', 96)
('IOWA', 100)
('MAINE', 100)
('CALIFORNIA', 100)
('ARKANSAS', 100)
('MISSISSIPPI', 95)
('RHODE ISLAND', 96)
('PENNSYLVANIA', 100)
('IDAHO', 100)
('TENNESSEE', 94)
('NORTH DAKOTA', 96)
('IOWA', 100)
('RHODE ISLAND', 96)
('DELAWARE', 100)
Before Correct State:
 0                Texas
1        NorthCarolina
2                 Iowa
3                Maine
4           California
5             Arkansas
6           Mississipi
7          RhodeIsland
8         Pennsylvania
9                Idaho
10            Tenessee
11         NorthDakota
12                Iowa
13         RhodeIsland
14            Delaware
Total                0
Name: s